In [116]:
#/python/Item_CF.py
# -*- coding: UTF-8 -*-
from math import sqrt
import operator
import pandas as pd
#1.构建用户-->物品的倒排
def loadData(files):
    data ={}
    for line in files:
        user,score,item=line.split(",")
        data.setdefault(user,{})
        data[user][item]=score
    print("----1.用户：物品的倒排----")
    print(data)
    return data

def loadData2(files):
    data={}
    for line in files:
        user,item,score,timestamp=line.split(",")
        data.setdefault(user,{})
        data[user][item]=score
    print("----1.用户：物品的倒排----")
    print(data)
    return data

#2.计算
# 2.1 构造物品-->物品的共现矩阵
# 2.2 计算物品与物品的相似矩阵
#(这里采用的是余弦相似度算法计算的物品间的相似度)
def similarity(data):
    # 2.1 构造物品：物品的共现矩阵
    N={}#喜欢物品i的总人数
    C={}#喜欢物品i也喜欢物品j的人数
    for user,item in data.items():
        for i,score in item.items():
            N.setdefault(i,0)
            N[i]+=1
            C.setdefault(i,{})
            for j,scores in item.items():
                if j not in i:
                    C[i].setdefault(j,0)
                    C[i][j]+=1
                    
    print("---2.构造的共现矩阵---")
    print ('进行i行为的总人数:',N)
    print ('进行i行为也进行j行为的人数:',C)

    #2.2 计算物品与物品的相似矩阵
    W={}
    for i,item in C.items():
        W.setdefault(i,{})
        for j,item2 in item.items():
            W[i].setdefault(j,0)
            W[i][j]=C[i][j]/sqrt(N[i]*N[j])
            
    print("---3.构造的相似矩阵---")
    print(W)
    return W,N,C

#3.根据用户的历史记录，给用户推荐物品
def recommandList(data,W,user,k=3,N=10):
    rank={}
    for i,score in data[user].items():	#获得用户user历史记录，如A用户的历史记录为{'a': '1', 'b': '1', 'd': '1'}
        for j,w in sorted(W[i].items(),key=operator.itemgetter(1),reverse=True)[0:k]:	#获得与物品i相似的k个物品
            if j not in data[user].keys():	#该相似的物品不在用户user的记录里
                rank.setdefault(j,0)
                rank[j]+=float(score) * w
                
    print("---4.推荐----")
    print(sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[0:N])
    return sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[0:N]

In [117]:
df = pd.DataFrame([['A','a',1],['A','b',2],['B','c',3],['B','a',4],['C','b',5],['D','c',6],['E','a',7],['E','c',8],
                   ['A','e',9],['F','b',10],['D','e',11],['F','e',12],['B','a',13]],columns=['user','behaviors','id'])

In [118]:
df

,user,behaviors,id
0,A,a,1
1,A,b,2
2,B,c,3
3,B,a,4
4,C,b,5
5,D,c,6
6,E,a,7
7,E,c,8
8,A,e,9
9,F,b,10


In [119]:
bhs_num = df.groupby('user')['behaviors'].value_counts()
bhslis= {}
for i in set(df['user']):
    bhs = bhs_num[i].to_dict()
    bhslis[i] = bhs
print('---1.构造行为倒排表---')
print(bhslis)

---1.构造行为倒排表---
{'C': {'b': 1}, 'B': {'a': 2, 'c': 1}, 'A': {'a': 1, 'b': 1, 'e': 1}, 'F': {'b': 1, 'e': 1}, 'E': {'a': 1, 'c': 1}, 'D': {'c': 1, 'e': 1}}


In [120]:
bhslis

{'C': {'b': 1},
 'B': {'a': 2, 'c': 1},
 'A': {'a': 1, 'b': 1, 'e': 1},
 'F': {'b': 1, 'e': 1},
 'E': {'a': 1, 'c': 1},
 'D': {'c': 1, 'e': 1}}

In [121]:
W,N,C=similarity(bhslis)

---2.构造的共现矩阵---
进行i行为的总人数: {'b': 3, 'a': 3, 'c': 3, 'e': 3}
进行i行为也进行j行为的人数: {'b': {'a': 1, 'e': 2}, 'a': {'c': 2, 'b': 1, 'e': 1}, 'c': {'a': 2, 'e': 1}, 'e': {'a': 1, 'b': 2, 'c': 1}}
---3.构造的相似矩阵---
{'b': {'a': 0.3333333333333333, 'e': 0.6666666666666666}, 'a': {'c': 0.6666666666666666, 'b': 0.3333333333333333, 'e': 0.3333333333333333}, 'c': {'a': 0.6666666666666666, 'e': 0.3333333333333333}, 'e': {'a': 0.3333333333333333, 'b': 0.6666666666666666, 'c': 0.3333333333333333}}


In [6]:
recommandList(data,W,'A',3,10)	#推荐

---4.推荐----
[('c', 1.2440169358562925), ('e', 0.5773502691896258)]


[('c', 1.2440169358562925), ('e', 0.5773502691896258)]